In [1]:
%matplotlib inline

# Modules
import datetime as dt
import matplotlib as mp
from statsmodels.graphics.tsaplots import plot_acf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, GaussianNoise, GRU, LSTM, Conv1D
from keras.layers.pooling import MaxPooling1D, GlobalAveragePooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers.wrappers import Bidirectional
import keras
import os
import pandas as pd

# Source data
data_file_path = ''
data_filename = 'spx_history.csv'

# Model
model_file = 'model.hd5'
num_epochs = 10000
validation_frac = 0.2
batch_size = 128
optimizer = 'adam'
loss = 'binary_crossentropy'
metrics = ['accuracy']

# Other config
dt_format = '%Y-%m-%d'
window = 2500 # days
investment_horizon = 250 # days
stride = 1 #days

/home/aind2/anaconda3/envs/aind2/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


In [2]:
def get_price_data(filename, dt_format):
    prices = pd.read_csv(filename,
                         delimiter=',',
                         header=0,
                         names=['date', 'P_close'],
                         index_col=0,
                         parse_dates=True,
                         date_parser=lambda date_str: dt.datetime.strptime(date_str, dt_format))
    return prices

In [3]:
def get_processed_data(prices_df, investment_horizon):
    prices_df['log_P'] = prices_df['P_close'].apply(np.log)
    prices_df['diff'] = prices_df['log_P'].diff(1)
    midpoint = (prices_df['diff'].max() + prices_df['diff'].min())/2
    scale = (prices_df['diff'].max() - prices_df['diff'].min())/2
    prices_df['scaled'] = (prices_df['diff'] - midpoint)/scale

    def map_outcome(x):
        if x == False:
            return 0
        elif x == True:
            return 1
        else:
            return np.nan
    
    prices_df['return'] = prices_df['log_P'].diff(investment_horizon)
    prices_df['outcome'] = (prices_df['return'] > prices_df['diff'].median()).map(map_outcome)
    
    print(prices_df.head())
    print(prices_df.describe())

    return prices_df[['scaled', 'outcome']].dropna(), midpoint, scale

In [4]:
def get_samples(data_df, window, stride):
    time_series = data_df.iloc[:, 0].values
    outcomes = data_df.iloc[:, 1].values
    x, y = zip(*[(time_series[i-window:i], outcomes[i]) for i in range(window, len(time_series), stride)])
    
    return np.array(x).reshape(-1, window, 1), np.array(y).reshape(-1, 1, 1)

In [5]:
def get_model():
    dropout = 0.5
    model = keras.models.Sequential()
    
    # Convolutions
    model.add(Conv1D(32, 2, padding='same', activation='relu', input_shape=(None,1)))
    model.add(MaxPooling1D(2, padding='same'))
    model.add(Dropout(dropout))
    model.add(Conv1D(64, 2, padding='same', activation='relu'))
    model.add(MaxPooling1D(2, padding='same'))
    model.add(Dropout(dropout))
    model.add(Conv1D(128, 2, padding='same', activation='relu'))
    model.add(MaxPooling1D(2, padding='same'))
    model.add(Dropout(dropout))
    model.add(Conv1D(128, 2, padding='same', activation='relu'))
    model.add(MaxPooling1D(2, padding='same'))
    model.add(Dropout(dropout))
#     model.add(Conv1D(128, 2, padding='same', activation='relu'))
#     model.add(MaxPooling1D(2, padding='same'))
#     model.add(Dropout(dropout))
#     model.add(Conv1D(128, 2, padding='same', activation='relu'))
#     model.add(MaxPooling1D(2, padding='same'))
#     model.add(Dropout(dropout))
#     model.add(Conv1D(128, 2, padding='same', activation='relu'))
#     model.add(MaxPooling1D(2, padding='same'))
#     model.add(Dropout(dropout))
    
#     # Recurrents
#     model.add(GRU(128, return_sequences=True, go_backwards=True))
#     model.add(Dropout(dropout))
#     model.add(GRU(128, return_sequences=False, go_backwards=False))
#     model.add(Dropout(dropout))
    
    # Dense for final prediction
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(4, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(2, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [6]:
full_filename = os.path.join(data_file_path, data_filename)
data = get_price_data(full_filename, dt_format)
processed_data, midpoint, scale = get_processed_data(data, investment_horizon)
X, y = get_samples(processed_data, window, stride)

            P_close     log_P      diff    scaled  return  outcome
date                                                              
1927-12-30    17.66  2.871302       NaN       NaN     NaN        0
1928-01-03    17.76  2.876949  0.005647  0.226387     NaN        0
1928-01-04    17.72  2.874694 -0.002255  0.185090     NaN        0
1928-01-05    17.55  2.865054 -0.009640  0.146491     NaN        0
1928-01-06    17.66  2.871302  0.006248  0.229532     NaN        0
            P_close         log_P          diff        scaled        return  \
count  22504.000000  22504.000000  22503.000000  22503.000000  22254.000000   
mean     399.850251      4.734416      0.000220      0.198023      0.053370   
std      573.245719      1.723863      0.011763      0.061478      0.203029   
min        4.400000      1.481605     -0.228997     -1.000000     -1.223378   
25%       22.987500      3.134951     -0.004572      0.172979     -0.044455   
50%       96.700000      4.571613      0.000460      0.19

In [7]:
X.shape

(20003, 2500, 1)

In [8]:
model = get_model()
model.summary()
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, None, 32)          96        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 32)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 64)          4160      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 64)          0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         16512     
__________

In [9]:
checkpointer = ModelCheckpoint(filepath=model_file, monitor='val_loss', save_best_only=True, verbose=1)
earlystopper = EarlyStopping(monitor='val_loss', patience=10)

training_history = model.fit(X, y, batch_size=batch_size, epochs=num_epochs, verbose=1,
          callbacks=[checkpointer, earlystopper], validation_split=validation_frac)

Train on 16002 samples, validate on 4001 samples
Epoch 1/10000
16002/16002 [==============================] - 19s - loss: 0.6820 - acc: 0.6938 - val_loss: 0.6673 - val_acc: 0.7318
Epoch 2/10000
16002/16002 [==============================] - 12s - loss: 0.6635 - acc: 0.6965 - val_loss: 0.6476 - val_acc: 0.7318
Epoch 3/10000
16002/16002 [==============================] - 12s - loss: 0.6489 - acc: 0.6965 - val_loss: 0.6315 - val_acc: 0.7318
Epoch 4/10000
16002/16002 [==============================] - 12s - loss: 0.6379 - acc: 0.6965 - val_loss: 0.6192 - val_acc: 0.7318
Epoch 5/10000
16002/16002 [==============================] - 12s - loss: 0.6304 - acc: 0.6965 - val_loss: 0.6103 - val_acc: 0.7318
Epoch 6/10000
16002/16002 [==============================] - 12s - loss: 0.6249 - acc: 0.6965 - val_loss: 0.6034 - val_acc: 0.7318
Epoch 7/10000
16002/16002 [==============================] - 12s - loss: 0.6210 - acc: 0.6965 - val_loss: 0.5982 - val_acc: 0.7318
Epoch 8/10000
16002/16002 [=======

16002/16002 [==============================] - 12s - loss: 0.6138 - acc: 0.6965 - val_loss: 0.5843 - val_acc: 0.7318
Epoch 30/10000
16002/16002 [==============================] - 12s - loss: 0.6138 - acc: 0.6965 - val_loss: 0.5845 - val_acc: 0.7318
Epoch 31/10000
16002/16002 [==============================] - 12s - loss: 0.6138 - acc: 0.6965 - val_loss: 0.5846 - val_acc: 0.7318
Epoch 32/10000
16002/16002 [==============================] - 12s - loss: 0.6138 - acc: 0.6965 - val_loss: 0.5844 - val_acc: 0.7318
Epoch 33/10000
16002/16002 [==============================] - 12s - loss: 0.6138 - acc: 0.6965 - val_loss: 0.5846 - val_acc: 0.7318
Epoch 34/10000
16002/16002 [==============================] - 12s - loss: 0.6138 - acc: 0.6965 - val_loss: 0.5848 - val_acc: 0.7318
Epoch 35/10000
16002/16002 [==============================] - 12s - loss: 0.6138 - acc: 0.6965 - val_loss: 0.5848 - val_acc: 0.7318
Epoch 36/10000
16002/16002 [==============================] - 12s - loss: 0.6138 - acc: 0.6